In [1]:
# 1
import numpy as np
import cv2

green = (0,255,0)
red = (0,0,255)
white = (255,255,255)
fonts = cv2.FONT_HERSHEY_COMPLEX
cap = cv2.VideoCapture(0)

###################################################################

# 2. size of the frame
width = 960
height= 720
cap.set(3,height)
cap.set(4,width)

###################################################################

def empty(a):
    pass

###################################################################

# 5. windows and trackbars
cv2.namedWindow("Parameters")
cv2.createTrackbar("Threshold1","Parameters",70,255,empty)
cv2.createTrackbar("Threshold2","Parameters",70,255,empty)
cv2.createTrackbar("Area","Parameters",1000,10000,empty)

###################################################################

# 8. find contours function
def getContours(img,imgContour):
    
# 9. arguments: image, contour retrieval mode, contour approximation method
# outputs the contours and hierarchy
# Contours is a Python list of all the contours in the image
# Each contour is a Numpy array of (x,y) coordinates of boundary points of the object
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# 10    
    for cnt in contours:
# 11        
        area = cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Parameters")
# 12        
        if area > areaMin:
#             cv2.drawContours(imgContour,cnt,-1,red,7)
            peri = cv2.arcLength(cnt, True)
    
# 13. approximate the shape of polygonal curves to the specified precision
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
#             print(len(approx))

# 14. coordinates of top-left corner, width, height
            x,y,w,h = cv2.boundingRect(approx)
    
# for central coordinates
            if(x+w > 400 and 400 > x and y+h > 285 and 285 > y):
# draw rectangle
                cv2.rectangle(imgContour,(x,y),(x+w,y+h),green,3)
                
#                 cv2.putText(imgContour,"Points: " + str(len(approx)),(x+w+20,y+20),fonts,0.7,green,2)
#                 cv2.putText(imgContour,"Height: " + str(int(h)),(x+w+20,y),fonts,0.7,green,2)
#                 cv2.putText(imgContour,"Width: " + str(int(w)),(x+w+20,y+25),fonts,0.7,green,2)
#                 cv2.putText(imgContour,"Aspect: " + str(round((h/w),1)),(x+w+20,y+25),fonts,0.7,green,2)
#                 cv2.putText(imgContour,"Distance: " + str(int(distance)),(x+w+20,y+25),fonts,0.7,green,2)
#                 cv2.putText(imgContour,"Area: " + str(int(area)),(x+w+20,y+50),fonts,0.7,green,2)

                if(area < 8300):
                    cv2.putText(imgContour,"Pinch",(x,y+h+20),fonts,0.7,green,2)
                elif(area > 15000):
                    cv2.putText(imgContour,"Power",(x,y+h+20),fonts,0.7,green,2)
                else:
                    cv2.putText(imgContour,"Tripod",(x,y+h+20),fonts,0.7,green,2)
    
###################################################################

while(True):        
    success, img = cap.read()    # Capture frame-by-frame
    imgContour = img    
# 4
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)    
    
# 6. Canny edge detector is an edge detection operator that uses a multi-stage algorithm
# to detect a wide range of edges in images

# canny image takes 2 thresholds (upper and lower)
# If a pixel gradient is higher than the upper threshold, it is accepted as an edge.
# If it is between the two thresholds, it will be accepted only if it is connected
# to a pixel that is above the upper threshold.

# get trackbar position from the parameters window
    threshold1 = cv2.getTrackbarPos("Threshold1","Parameters")
    threshold2 = cv2.getTrackbarPos("Threshold2","Parameters")
    imgCanny = cv2.Canny(imgGray,threshold1,threshold2)
    
# 7. dilation function to remove overlaps and noise    
    kernel = np.ones((5,5))
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
    
# contour: outline representing or bounding the shape    
    getContours(imgDil, imgContour)
    
#     imgStack = stackImages(0.8,([img, imgBlur, imgGray], [imgDil,imgDil,imgDil]))
# 3
    cv2.imshow('frame',imgContour)
    
###################################################################

    if cv2.waitKey(1)==27: break    # esc

cap.release()
cv2.destroyAllWindows()